<a href="https://colab.research.google.com/github/gowen100/Machine-Learning/blob/master/TensorFlow_No_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [22]:
#set up multivariate linear regression example data 

#define the dataset size
num_features = 6
num_samples = 10000

#define dataset parameters 
model_a = np.random.normal(size=(num_features))
model_c =np.random.normal()

print("model a is {}".format(model_a))
print("model c is {}".format(model_c))

# create some random initial values and then work out the result adding in some random noise
x_train = np.random.normal(size=(num_samples,num_features))*100
y_train = x_train @ model_a + model_c + np.random.normal(size= num_samples)


model a is [ 0.76768127 -0.50678936  0.47532782  0.43127988  0.12477923 -0.1184996 ]
model c is 0.2644701633703223


In [23]:
#use tensorflow to compute gradients and optimise the fit

a = tf.Variable(tf.zeros(shape=(num_features,)),name="a", trainable=True, dtype=tf.float32)
c = tf.Variable(tf.zeros(shape=(1,)),name ="c", trainable=True, dtype=tf.float32)

@tf.function
def f(x):
    return tf.tensordot(x,a,axes=1)+c

#print(tf.autograph.to_code(f.python_function))

variables = [a,c]

optimizer = tf.optimizers.Adam(0.005)
loss_object = tf.keras.losses.MeanSquaredError()

for i in range(1000):
    with tf.GradientTape() as tape:
        y_pred = f(tf.cast(x_train,tf.float32))
        loss = loss_object(tf.cast(y_train,tf.float32),y_pred)
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        if i %100 ==0: 
            print('Epoch: {} Loss: {}'.format(i,loss.numpy()))

Epoch: 0 Loss: 13011.8203125
Epoch: 100 Loss: 1409.5523681640625
Epoch: 200 Loss: 106.43506622314453
Epoch: 300 Loss: 5.155116558074951
Epoch: 400 Loss: 1.0517330169677734
Epoch: 500 Loss: 0.982807993888855
Epoch: 600 Loss: 0.9822824001312256
Epoch: 700 Loss: 0.9822757840156555
Epoch: 800 Loss: 0.982275664806366
Epoch: 900 Loss: 0.9822757840156555


In [24]:
print('Model fit:',a.numpy(),c.numpy())
print('True :    ',model_a,[model_c])

Model fit: [ 0.76783264 -0.506731    0.47550273  0.43135658  0.12464958 -0.11853882] [0.26828253]
True :     [ 0.76768127 -0.50678936  0.47532782  0.43127988  0.12477923 -0.1184996 ] [0.2644701633703223]


In [25]:
# statsmodels OLS .. remember to add in the constant
import statsmodels.api as sm

model = sm.OLS(y_train, sm.add_constant(x_train)).fit()
print(model.summary(),model.params)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.205e+07
Date:                Fri, 29 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:11:56   Log-Likelihood:                -14100.
No. Observations:               10000   AIC:                         2.821e+04
Df Residuals:                    9993   BIC:                         2.826e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2683      0.010     27.053      0.0

In [26]:
#SKlearn linear model

from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x_train,y_train)
vars(regr)
print('Model fit :',regr.coef_,regr.intercept_)
print('True :     ',model_a,model_c)
vars(regr)

Model fit : [ 0.76783326 -0.50673142  0.47550265  0.43135642  0.12464959 -0.11853882] 0.2682801086081552
True :      [ 0.76768127 -0.50678936  0.47532782  0.43127988  0.12477923 -0.1184996 ] 0.2644701633703223


{'_residues': 9822.754648000235,
 'coef_': array([ 0.76783326, -0.50673142,  0.47550265,  0.43135642,  0.12464959,
        -0.11853882]),
 'copy_X': True,
 'fit_intercept': True,
 'intercept_': 0.2682801086081552,
 'n_jobs': None,
 'normalize': False,
 'rank_': 6,
 'singular_': array([10214.53788908, 10192.06087552, 10096.87001062,  9989.27379434,
         9933.06566934,  9843.73287731])}

In [27]:
#optimise loss function. Need to combine a+c into the first parameter 

from scipy.optimize import minimize

def cost(a,x,y):
    predict = a[-1] + x@a[:-1]
    return np.mean((predict-y)**2)

opt_result = minimize(cost, np.zeros(num_features+1), args=(x_train,y_train),method='BFGS', options={'maxiter': 5000})
print('Model fit :',opt_result.x[:-1],opt_result.x[-1])
print('True :     ',model_a,model_c)


Model fit : [ 0.76783325 -0.50673143  0.47550264  0.43135641  0.12464959 -0.11853883] 0.2682800907212365
True :      [ 0.76768127 -0.50678936  0.47532782  0.43127988  0.12477923 -0.1184996 ] 0.2644701633703223
